### Load Data

In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [3]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')


In [6]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [7]:
ratings = ratings.drop('timestamp', axis=1)

In [8]:
ratings.shape

(100836, 3)

In [9]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
new = pd.merge(links, movies, on='movieId')

In [11]:
new_2 = pd.merge(new, ratings, on='movieId')

In [12]:
new_3 = pd.merge(new_2, tags, on='movieId')

### Merge Data Frames

In [13]:
# merge links and movie
#frames_1 = [links, movies]
#new = pd.concat(frames_1)


In [14]:
# merge new and ratings
#frames_2 = [new, ratings]
#new_2 = pd.concat(frames_2)

In [15]:
# Drop movieId and timestamp
#tags_2 = tags.drop(columns=['movieId', 'timestamp'])

In [16]:
# concat new_2 and tags
#frames = [new_2, tags_2]
#new_3 = pd.concat(frames)
#new_3 = pd.merge(new_2, tags)


In [17]:
new_3.shape

(233213, 10)

### Preprocessing

In [18]:
# Check for null values
new_3.isnull().sum()

movieId      0
imdbId       0
tmdbId       0
title        0
genres       0
userId_x     0
rating       0
userId_y     0
tag          0
timestamp    0
dtype: int64

In [19]:
new_3['movieId'].nunique()

1554

In [20]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [21]:
#df = new_3.drop_duplicates(subset=['movieId'])

In [22]:
new_3

,movieId,imdbId,tmdbId,title,genres,userId_x,rating,userId_y,tag,timestamp
0,1,114709,862.0,Toy Story (1995),"Adventure , Animation , Children , Comedy , Fa...",1,4.0,336,pixar,1139045764
1,1,114709,862.0,Toy Story (1995),"Adventure , Animation , Children , Comedy , Fa...",1,4.0,474,pixar,1137206825
2,1,114709,862.0,Toy Story (1995),"Adventure , Animation , Children , Comedy , Fa...",1,4.0,567,fun,1525286013
3,1,114709,862.0,Toy Story (1995),"Adventure , Animation , Children , Comedy , Fa...",5,4.0,336,pixar,1139045764
4,1,114709,862.0,Toy Story (1995),"Adventure , Animation , Children , Comedy , Fa...",5,4.0,474,pixar,1137206825
...,...,...,...,...,...,...,...,...,...,...
233208,187595,3778644,348350.0,Solo: A Star Wars Story (2018),"Action , Adventure , Children , Sci-Fi",586,5.0,62,star wars,1528934552
233209,193565,1636780,71172.0,Gintama: The Movie (2010),"Action , Animation , Comedy , Sci-Fi",184,3.5,184,anime,1537098582
233210,193565,1636780,71172.0,Gintama: The Movie (2010),"Action , Animation , Comedy , Sci-Fi",184,3.5,184,comedy,1537098587
233211,193565,1636780,71172.0,Gintama: The Movie (2010),"Action , Animation , Comedy , Sci-Fi",184,3.5,184,gintama,1537098603


In [23]:
# convert df to a csv file
new_3.to_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Explode genres column

In [24]:
# split genre column based on ,
new_3['new_genres'] = new_3['genres'].str.split(',', 10)

In [25]:
# explode new_genres
df = new_3.explode('new_genres')

In [26]:
# drop genres from df
df_2 = df.drop(columns=['genres'])

In [27]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


### OHE title

In [28]:
one_hot = pd.get_dummies(df_2['title'])

In [29]:
df = df_2.drop('title',axis = 1)

In [30]:
df = pd.concat([df, one_hot], axis=1)

In [31]:
df.shape

(765271, 1563)

In [32]:
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1554 entries, (500) Days of Summer (2009) to eXistenZ (1999)
dtypes: uint8(1554)
memory usage: 1.1 GB


### OHE tag

In [33]:
one_hot_tag = pd.get_dummies(df['tag'])

In [34]:
df = df.drop('tag',axis = 1)

In [35]:
df = pd.concat([df, one_hot_tag], axis=1)

In [36]:
one_hot_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1584 entries, "artsy" to zombies
dtypes: uint8(1584)
memory usage: 1.1 GB


### OHE new_genres

In [37]:
one_hot_genre = pd.get_dummies(df['new_genres'])

In [38]:
df =df.drop('new_genres',axis = 1)

In [39]:
df = pd.concat([df, one_hot_genre], axis=1)

### Base Line Model

In [40]:
from sklearn.neighbors import NearestNeighbors

In [41]:
#knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
#knn.fit(df)

### Suprise

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 3209 entries, movieId to Western
dtypes: float64(2), int64(5), uint8(3202)
memory usage: 2.3 GB


### Change Data Frames to suprise DataFrame

In [43]:
import surprise

In [44]:
from surprise.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [45]:
ratings.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [46]:
ratings.shape

(100836, 3)

In [47]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [48]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [49]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [50]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [51]:
df.head()

,movieId,imdbId,tmdbId,userId_x,rating,userId_y,timestamp,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),...,Musical,Musical,Mystery,Mystery,Romance,Romance,Sci-Fi,Sci-Fi,Thriller,Western
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [53]:
# from sklearn.model_selection import train_test_split

In [54]:
trainset, testset = train_test_split(data, test_size=0.2)

In [55]:
#trainset.head()

### KNNS Basic (Best RMSE=.5)

In [56]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [57]:
basic = knns.KNNBasic()
basic.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [58]:
predictions = basic.test(testset)

In [59]:
print(accuracy.rmse(predictions))

RMSE: 1.1608
1.1607789399132211


In [60]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1614
1.1614321499850797


### Matrix Factorization

In [61]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

### Suprise with grid search

In [62]:
from sklearn.neighbors import KNeighborsClassifier

In [63]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
               'reg_all': [0.4, 0.6]}

In [64]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 26.5min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 27.6min finished


In [65]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1569
1.1568838777585888


### Predictions SVD

In [66]:
user_34_prediction = svd.predict('12', '85')
user_34_prediction

Prediction(uid='12', iid='85', r_ui=None, est=4.0, details={'was_impossible': False})

In [67]:
user_34_prediction[3]

4.0

In [68]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
def get_Ui(iid):
    """
args:
iid: the raw id of the item
returns:
the number of users that have rated the item.
"""
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predictions['Iu'] = df_predictions.uid.apply(get_Iu)
df_predictions['Ui'] = df_predictions.iid.apply(get_Ui)
df_predictions['err'] = abs(df_predictions.est - df_predictions.rui)

In [69]:
best_predictions = df_predictions.sort_values(by='err')[:10]
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
6338,62,111360,4.0,4.0,{'was_impossible': False},286,10,0.0
17556,45,3273,4.0,4.0,{'was_impossible': False},308,23,0.0
9525,606,937,4.0,4.0,{'was_impossible': False},903,1,0.0
9527,200,53125,4.0,4.0,{'was_impossible': False},272,46,0.0
9528,262,230,4.0,4.0,{'was_impossible': False},45,17,0.0
4313,367,3189,4.0,4.0,{'was_impossible': False},136,9,0.0
9530,418,2005,4.0,4.0,{'was_impossible': False},75,51,0.0
14514,62,116823,4.0,4.0,{'was_impossible': False},286,19,0.0
4309,101,223,4.0,4.0,{'was_impossible': False},45,81,0.0
14520,91,903,4.0,4.0,{'was_impossible': False},457,48,0.0


### Graph Lab Model

In [70]:
# Make a train-test split
#train_data, validate_data = df.random_split(0.8)

### Cosine Similarity

In [71]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [72]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [73]:
#knn.train(trainset)

### Surprise Baseline Only

In [74]:
from surprise import BaselineOnly

In [75]:
# showcasing the cross_validate function as well 
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1531  1.1433  1.1497  1.1551  1.1442  1.1491  0.0047  
MAE (testset)     0.8388  0.8336  0.8423  0.8435  0.8316  0.8380  0.0047  
Fit time          0.21    0.23    0.23    0.24    0.23    0.23    0.01    
Test time         0.07    0.61    0.08    0.08    0.08    0.18    0.21    


{'test_rmse': array([1.15309612, 1.14328624, 1.1496584 , 1.15514496, 1.1442339 ]),
 'test_mae': array([0.83881652, 0.83357108, 0.84227458, 0.84347534, 0.83162961]),
 'fit_time': (0.21462702751159668,
  0.23091673851013184,
  0.23385095596313477,
  0.23668718338012695,
  0.23317193984985352),
 'test_time': (0.07447695732116699,
  0.6072878837585449,
  0.08085823059082031,
  0.07913804054260254,
  0.0787820816040039)}

### KNN with GridSearch

In [76]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

In [77]:

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [2, 3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs_1 = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs_1.fit(data)

print(gs_1.best_score["rmse"])
print(gs_1.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

### Grid Search

In [78]:
# param_grid = {
        #"n_epochs": [5, 10, 15, 20, 30, 40, 50, 100],
        # "lr_all": [0.001, 0.002, 0.005],
       # "reg_all": [0.02, 0.08, 0.4, 0.6]
#}

# smaller grid for testing
param_grid = {
    "n_epochs": [15, 20, 25],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 1.1492112897576199
BEST MAE: 	 0.8375377686160789
BEST params: 	 {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [79]:
def get_movie_id(movie_title):
    
    # Filter Movies dataframe for movie_title
    movie_title_df = movies[['title', 'movieId']]
    # Get corresponding id from row
    #movie_id = movie_title_df.query('title==movie_title')['movieId']
    movie_id = movie_title_df.loc['movieId']
    # Return movie_id of movie_title
    return movie_id

In [80]:
#movies = movies.set_index('title')

In [81]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


# Make the predictions

In [82]:
def get_movie_id(movie_title):
    
    # Filter Movies dataframe for movie_title
    movie_title_df = movies[['title', 'movieId']]
    # Get corresponding id from row
    #movie_id = movie_title_df.query('title==movie_title')['movieId']
  #  movie_id = movie_title_df['movieId']
    movie_id = movies[movies["title"] == movie_title]['movieId'].values[0]

    # Return movie_id of movie_title
    return movie_id


In [83]:
get_movie_id("Waiting to Exhale (1995)")

4

In [84]:
t = pd.DataFrame([[1, 5, 4]], columns=['u', 'i', 'rating'])
t

,u,i,rating
0,1,5,4


# Get recomended movies

In [85]:
info = {'u':[1],
       'i':[5],
       'rating':[4]}
data = surprise.Dataset.load_from_df(t, reader)

In [86]:
gs(data)

TypeError: 'GridSearchCV' object is not callable

In [ ]:
def model_function(u, i, rating):
#  user ranks 5 movies
    info = {'u':[u],
       'i':[i],
       'rating':[rating]}
# convert to suprise df
    data = surprise.Dataset.load_from_df(info, reader)
    print(data)
# run through model
    gs(data)


In [ ]:
model_function(1,5,4)

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
#trainset = data.build_full_trainset()
#algo = SVD()
#algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
#predictions = algo.test(testset)

#top_n = get_top_n(predictions, n=10)

 #Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
get_top_n('Toy Story (1995)')

In [ ]:
movies_id = get_movie_id('Father of the Bride Part II (1995)')
# pass movie_id and a user_id to model to make recs
gs.test(testset)
gs.predict()

In [ ]:
get_movie_id('Father of the Bride Part II (1995)')

In [ ]:
#top_n('Toy Story 1995')

In [ ]:
#get_top_n('Toy Story (1995)')
movies.head()

### Movie Reccomendation Model

In [ ]:
df_2

In [ ]:
df_2.head()

In [ ]:
import RAKE
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_3 = df_2[['title','new_genres','tag','rating']]
df_3.head()

In [ ]:
# initializing the new column
df_3['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['tag']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df_3.drop(columns = ['tag'], inplace = True)

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['tag'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Toy Story (1995)')

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

### My Own Algorythem

In [ ]:
ratings.head(1)

In [ ]:
from surprise import AlgoBase
from surprise.model_selection import cross_validate


class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def estimate(self, u, i):

        return 3


#data = Dataset.load_builtin('ml-100k')
algo = MyOwnAlgorithm()

cross_validate(algo, data, verbose=True)

In [ ]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def fit(self, trainset):

        # Here again: call base method before doing anything.
        AlgoBase.fit(self, trainset)

        # Compute the average rating. We might as well use the
        # trainset.global_mean attribute ;)
        self.the_mean = np.mean([r for (_, _, r) in
                                 self.trainset.all_ratings()])

        return self

    def estimate(self, u, i):

        return self.the_mean

In [ ]:
    def estimate(self, u, i):

        sum_means = self.trainset.global_mean
        div = 1

        if self.trainset.knows_user(u):
            sum_means += np.mean([r for (_, r) in self.trainset.ur[u]])
            div += 1
        if self.trainset.knows_item(i):
            sum_means += np.mean([r for (_, r) in self.trainset.ir[i]])
            div += 1

        return sum_means / div

In [ ]:
from surprise import PredictionImpossible

In [ ]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options,
                          bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # ... Aaaaand return the baseline estimate anyway ;)

In [ ]:
import inspect

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for userId, movieId, rating in predictions.items():
        top_n[userId].append((movieId, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for userId, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[userId] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
get_top_n('Toy Story (1995)')

In [ ]:
ratings

In [ ]:
movies

In [ ]:
#get_top_n('Toy Story (1995)')

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    #top_n = defaultdict(list)
    #for uid, iid, true_r, est, _ in predictions:
        #top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])